In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Step 1: กำหนดโมเดล Pydantic สำหรับผลลัพธ์ที่มีโครงสร้าง

โมเดลเหล่านี้กำหนด **สคีมา** ที่เอเย่นต์จะส่งคืน การใช้ `response_format` กับ Pydantic ช่วยให้มั่นใจว่า:
- ✅ การดึงข้อมูลที่ปลอดภัยตามประเภท
- ✅ การตรวจสอบอัตโนมัติ
- ✅ ไม่มีข้อผิดพลาดในการแยกวิเคราะห์จากการตอบข้อความฟรี
- ✅ การส่งเสริมเส้นทางตามเงื่อนไขที่ง่ายขึ้นโดยอิงจากฟิลด์


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Step 2: สร้างเครื่องมือจองโรงแรม

เครื่องมือนี้คือสิ่งที่ **availability_agent** จะเรียกใช้เพื่อตรวจสอบว่าห้องพักว่างหรือไม่ เราใช้ตัวตกแต่ง `@ai_function` เพื่อ:
- แปลงฟังก์ชัน Python เป็นเครื่องมือที่สามารถเรียกใช้โดย AI
- สร้างสคีมา JSON อัตโนมัติสำหรับ LLM
- จัดการการตรวจสอบค่าพารามิเตอร์
- เปิดใช้งานการเรียกใช้โดยอัตโนมัติโดยเอเยนต์

สำหรับการสาธิตนี้:
- **สตอกโฮล์ม, ซีแอตเทิล, โตเกียว, ลอนดอน, อัมสเตอร์ดัม** → มีห้องพัก ✅
- **เมืองอื่นๆ ทั้งหมด** → ไม่มีห้องพัก ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## ขั้นตอนที่ 3: กำหนดฟังก์ชันเงื่อนไขสำหรับการกำหนดเส้นทาง

ฟังก์ชันเหล่านี้จะตรวจสอบการตอบกลับของตัวแทนและกำหนดว่าจะเลือกเส้นทางใดในกระบวนการทำงาน

**รูปแบบสำคัญ:**
1. ตรวจสอบว่าข้อความเป็น `AgentExecutorResponse` หรือไม่
2. แยกวิเคราะห์ผลลัพธ์ที่มีโครงสร้าง (โมเดล Pydantic)
3. ส่งคืน `True` หรือ `False` เพื่อควบคุมการกำหนดเส้นทาง

กระบวนการทำงานจะประเมินเงื่อนไขเหล่านี้บน **ขอบเขต** เพื่อกำหนดว่าจะเรียกตัวดำเนินการใดต่อไป


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Step 4: สร้างตัวดำเนินการแสดงผลแบบกำหนดเอง

ตัวดำเนินการเป็นส่วนประกอบของเวิร์กโฟลว์ที่ทำการแปลงข้อมูลหรือผลข้างเคียง เราใช้ตัวตกแต่ง `@executor` เพื่อสร้างตัวดำเนินการแบบกำหนดเองที่แสดงผลลัพธ์สุดท้าย

**แนวคิดสำคัญ:**
- `@executor(id="...")` - ลงทะเบียนฟังก์ชันเป็นตัวดำเนินการเวิร์กโฟลว์
- `WorkflowContext[Never, str]` - การบอกชนิดข้อมูลสำหรับอินพุต/เอาต์พุต
- `ctx.yield_output(...)` - ส่งผลลัพธ์เวิร์กโฟลว์สุดท้าย


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Step 5: โหลดตัวแปรสิ่งแวดล้อม

กำหนดค่าลูกค้า LLM ตัวอย่างนี้ทำงานกับ:
- **GitHub Models** (ระดับฟรีด้วยโทเค็น GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## ขั้นตอนที่ 6: สร้างเอเย่นต์ AI พร้อมผลลัพธ์ที่มีโครงสร้าง

เราสร้าง **เอเย่นต์เฉพาะทางสามตัว** โดยแต่ละตัวถูกห่อหุ้มใน `AgentExecutor`:

1. **availability_agent** - ตรวจสอบการว่างของโรงแรมโดยใช้เครื่องมือ
2. **alternative_agent** - เสนอเมืองทางเลือก (เมื่อไม่มีห้องว่าง)
3. **booking_agent** - ส่งเสริมการจอง (เมื่อมีห้องว่าง)

**คุณสมบัติที่สำคัญ:**
- `tools=[hotel_booking]` - มอบเครื่องมือให้กับเอเย่นต์
- `response_format=PydanticModel` - บังคับให้ผลลัพธ์เป็น JSON ที่มีโครงสร้าง
- `AgentExecutor(..., id="...")` - ห่อหุ้มเอเย่นต์เพื่อใช้ในเวิร์กโฟลว์


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## ขั้นตอนที่ 7: สร้างเวิร์กโฟลว์ด้วยขอบเชื่อมโยงตามเงื่อนไข

ตอนนี้เราใช้ `WorkflowBuilder` เพื่อสร้างกราฟด้วยการกำหนดเส้นทางตามเงื่อนไข:

**โครงสร้างเวิร์กโฟลว์:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**เมธอดสำคัญ:**
- `.set_start_executor(...)` - กำหนดจุดเริ่มต้น
- `.add_edge(from, to, condition=...)` - เพิ่มขอบเชื่อมโยงตามเงื่อนไข
- `.build()` - ทำงานให้เวิร์กโฟลว์เสร็จสมบูรณ์


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Step 8: Run Test Case 1 - City WITHOUT Availability (Paris)

มาทดสอบเส้นทาง **ไม่มีห้องว่าง** โดยขอโรงแรมในปารีส (ซึ่งไม่มีห้องว่างในระบบจำลองของเรา)


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Step 9: Run Test Case 2 - City WITH Availability (Stockholm)

ตอนนี้เรามาทดสอบเส้นทาง **availability** โดยการขอโรงแรมในสต็อกโฮล์ม (ซึ่งมีห้องว่างในจำลองของเรา)


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## ข้อสรุปที่สำคัญและขั้นตอนถัดไป

### ✅ สิ่งที่คุณได้เรียนรู้:

1. **รูปแบบ WorkflowBuilder**
   - ใช้ `.set_start_executor()` เพื่อกำหนดจุดเริ่มต้น
   - ใช้ `.add_edge(from, to, condition=...)` สำหรับการเส้นทางแบบมีเงื่อนไข
   - เรียก `.build()` เพื่อสรุป workflow

2. **การเส้นทางแบบมีเงื่อนไข**
   - ฟังก์ชันเงื่อนไขตรวจสอบ `AgentExecutorResponse`
   - วิเคราะห์ผลลัพธ์ที่มีโครงสร้างเพื่อทำการตัดสินใจเส้นทาง
   - คืนค่า `True` เพื่อเปิดใช้งานเส้นทาง, `False` เพื่อข้ามเส้นทาง

3. **การรวมเครื่องมือ**
   - ใช้ `@ai_function` เพื่อแปลงฟังก์ชัน Python เป็นเครื่องมือ AI
   - ตัวแทนเรียกใช้เครื่องมือโดยอัตโนมัติเมื่อจำเป็น
   - เครื่องมือคืนค่า JSON ที่ตัวแทนสามารถวิเคราะห์ได้

4. **ผลลัพธ์ที่มีโครงสร้าง**
   - ใช้โมเดล Pydantic เพื่อดึงข้อมูลที่ปลอดภัยตามประเภท
   - ตั้งค่า `response_format=MyModel` เมื่อสร้างตัวแทน
   - วิเคราะห์คำตอบด้วย `Model.model_validate_json()`

5. **ตัวดำเนินการกำหนดเอง**
   - ใช้ `@executor(id="...")` เพื่อสร้างส่วนประกอบ workflow
   - ตัวดำเนินการสามารถแปลงข้อมูลหรือทำผลข้างเคียง
   - ใช้ `ctx.yield_output()` เพื่อส่งผลลัพธ์ของ workflow

### 🚀 การใช้งานจริงในโลก:

- **การจองท่องเที่ยว**: ตรวจสอบความพร้อม, เสนอทางเลือก, เปรียบเทียบตัวเลือก
- **บริการลูกค้า**: เส้นทางตามประเภทปัญหา, ความรู้สึก, ลำดับความสำคัญ
- **อีคอมเมิร์ซ**: ตรวจสอบสต็อก, เสนอทางเลือก, ประมวลผลคำสั่งซื้อ
- **การควบคุมเนื้อหา**: เส้นทางตามคะแนนความเป็นพิษ, รายงานผู้ใช้
- **เวิร์กโฟลว์การอนุมัติ**: เส้นทางตามจำนวนเงิน, บทบาทผู้ใช้, ระดับความเสี่ยง
- **การประมวลผลหลายขั้นตอน**: เส้นทางตามคุณภาพข้อมูล, ความสมบูรณ์

### 📚 ขั้นตอนถัดไป:

- เพิ่มเงื่อนไขที่ซับซ้อนมากขึ้น (หลายเกณฑ์)
- ใช้วงจรซ้ำด้วยการจัดการสถานะ workflow
- เพิ่มเวิร์กโฟลว์ย่อยสำหรับส่วนประกอบที่ใช้ซ้ำได้
- รวมกับ API จริง (การจองโรงแรม, ระบบสต็อก)
- เพิ่มการจัดการข้อผิดพลาดและเส้นทางสำรอง
- แสดงภาพ workflow ด้วยเครื่องมือแสดงภาพในตัว


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษาอัตโนมัติ [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้ความถูกต้องสูงสุด โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่แม่นยำ เอกสารต้นฉบับในภาษาต้นทางถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลโดยมนุษย์มืออาชีพ เราจะไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดใดๆ ที่เกิดขึ้นจากการใช้การแปลนี้
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
